# Ad-Hoc

## 1. Run Ad-Hoc

### 1.1. List servers in group

In [ ]:
ansible group_debian --list

### 1.2. Work with module

```bash
$ ansible <group1, group2, ..., groupn | all> -m <module> -a "command string"
```

For example:
```bash
$ ansible all -m shell -a "ps aux"
```

## 2. Modules

### 2.1. Ping

In [ ]:
ansible group_debian1 -m ping

### 1.2. Raw 

In [ ]:
ansible group_debian1 -m raw -a "ps aux"

### 1.3. Software repository

#### 1.3.1. Yum

#### 1.3.2. Apt 

### 1.4. Template

- Arguments
    - `src`: template file in current machine
    - `dest`: target file in remote server
    - `owner`: owner of target file
    - `mode`: mode of target file, canbe numbers (like `0644`) or letters (like `u=rw,g=r,o=r`), see also `chmod` command
    - `backup`: `ture` if create backup file if file content is changed

In [ ]:
# Run template module
ansible group_debian1 -m template \
    -a "src='./test_template.txt' \
        dest='~/test_template.txt' \
        owner='alvin' \
        mode='u=rw,g=r,o=r' \
        backup='yes'" \
    --extra-vars "dynamic_name=Alvin";

# Show generated files on remote server
ansible group_debian1 -m raw -a "ls -al ~/test_template*";

# Show content of lastest generated file
ansible group_debian1 -a "cat ~/test_template.txt";

# remove all generated files on remote server
ansible group_debian1 -m raw -a "rm ~/test_template*";

### 1.5. Copy

- `src`: The source file on local machine, can be file or directory
- `content`: The content will be copied
- `dest`: The remote server to copied file
- `directory_mode`: The permission mode of remote directory
- `force`: Always copy file to target even file exist when value is 'yes'
- other arguments: all arguments for `file` module can use for this module, see `file` module

#### 1.5.1. Copy file

In [ ]:
# Use copy module
ansible group_debian1 -m copy -a "src='./ansible.cfg' dest='~/' force='yes'";

# Show copied file on remote server
ansible group_debian1 -m raw -a "ls -al ~/ansible.*";

# Remove copied files
ansible group_debian1 -m raw -a "rm ~/ansible.*";

#### 1.5.2. Copy files in folder

In [ ]:
# Use copy module
ansible group_debian1 -m copy -a "src='./hosts/' dest='~/target' force='yes'";

# Show copied folders
ansible group_debian1 -m raw -a "ls -al ~/target";

# Remove copied files
ansible group_debian1 -m raw -a "rm -rf ~/target*";

#### 1.5.3. Copy folder

In [ ]:
# Use copy module
ansible group_debian1 -m copy -a "src='./hosts' dest='~/target' force='yes'";

# Show copied folders
ansible group_debian1 -m raw -a "ls -al ~/target";
ansible group_debian1 -m raw -a "ls -al ~/target/hosts";

# Remove copied files
ansible group_debian1 -m raw -a "rm -rf ~/target*";

#### 1.5.4. Copy content

In [ ]:
# Use copy module
ansible group_debian1 -m copy -a "content='Hello World' dest='~/target.txt' force='yes'";

# Show copied folders
ansible group_debian1 -m raw -a "cat ~/target.txt";

# Remove copied files
ansible group_debian1 -m raw -a "rm target.txt";

### 1.6. User and Group

#### 1.6.1. User

- `home`: Home folder of user
- `groups`: Groups of user
- `uid`: Id of user
- `password`: Password of user, password cannot be plaintext, it need hashed by sha512
- `name`: User name
- `createhome`：`yes` if create home directory
- `system`: `yes` if a system user
- `remove`: `yes` if delete user home directory when `state = absent`
- `state`: Create (`statue=present`) or remove (`state=absent`) user
- `shell`: Which shell for this user
- `generate_ssh_key`: `yes` if create ssh key for this user
- `ssh_key_bits`: Length (bits) of ssh key
- `ssh_key_passphrase`: Phrase password of ssh key
- `ssh_key_file`: File name to save ssh key
- `ssh_key_type`: Type of ssh key

In [ ]:
# Add and modify user: if user exist, then modify this user by new arguments
ansible group_debian1_vault -m user \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "name='test' \
        password='{{ 'test'|password_hash('sha512') }}' \
        append='yes' \
        createhome='yes' \
        shell='/bin/bash' \
        generate_ssh_key='yes' \
        groups='sudo,root' \
        state='present'";
        
# Show user and password (hashed) on remote server
ansible group_debian1_vault -m raw \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "cat /etc/shadow" | grep 'test';
    
# Allow sudo for new user
ansible group_debian1_vault -m copy \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "content='alvin ALL=(ALL:ALL) ALL\n' \
        dest='/etc/sudoers.d/alvin' \
        force='yes' \
        mode='u=r,g=r'";
        
# Copy ssh public key from current machine to remove server
ansible group_debian1_vault -m authorized_key \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "user='test' \
        key='{{ lookup('file', '~/.ssh/id_rsa.pub') }}' \
        state='present'";
        
# Delete user
ansible group_debian1_vault -m user \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "user='test' \
        remove='yes' \
        state='absent'";

#### 1.6.2. Group

- `gid`：Id of group
- `name`: Name of group
- `state`: State of group, `present` or `absent`
- `system`: `yes` if a system group

In [ ]:
# Create a group
ansible group_debian1_vault -m group \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "name='test' \
        state='present'";
        
# Remove a group
ansible group_debian1_vault -m group \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "name='test' \
        state='absent'";

### 1.7. Service

- `arguments`: Arguments for commandline
- `enabled`: `yes` if start server when user login
- `name`: Name of service
- `runlevel`: Level of service
- `sleep`: If `state` is `restarted`, the delay seconds between `stop` and `start`
- `state`: State of service, value can be: `started`, `stopped`, `restarted`, `reloaded`

In [ ]:
# Stop service
ansible group_debian1_vault -m service \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "name='sshd' state='stopped'";
    
# Start service
ansible group_debian1_vault -m service \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "name='sshd' state='started'";

### 1.8. Get url

- `sha256sum`: sha256 checksum
- `timeout`: Timeout second of download
- `url`: Url of download file
- `url_password`, `url_username`: Password and username of download request
- `dest`: Filename to save download file, if just path name, use default filename
- `headers`: Headers of download request, use `key1: value1, key2: value2` form

In [ ]:
# Download file
ansible group_debian1_vault -m get_url \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "url='https://www.learningcontainer.com/download/sample-text-file/?wpdmdl=1669&refresh=5f918bb1200901603374001' \
        dest='~/test.txt' \
        timeout=20";
        
# Validate download file
ansible group_debian1_vault -m raw \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "cat ~/test.txt";
    
# Delete downloaded file
ansible group_debian1_vault -m raw \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "rm ~/test.txt";

### 1.9. Fetch

- `src`: Source file on remote service
- `dest`: Backup folder on local machine
- `flat`: `yes` if do not create full pathname like `<machine>/<full path>/<file>`, only transfer file to dest folder

In [ ]:
# Fetch remote file
ansible group_debian1_vault -m fetch \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "src='/home/alvin/.bashrc' dest='~/ansible_backup/' flat='yes'";
    
# Show backup file content
cat ~/ansible_backup/.bashrc;

# Delete fetched file
rm -rf ~/ansible_backup;

### 1.10. File

- `force`: `yes` means whatever file link exist, if symbol is exist, delete old symbol and create new one
- `group`: Define the group of file or directory
- `mode`: Define permissions of file or directory
- `owner`: Define the owner of file or directory
- `path`: Fullpath of file or directory to operate
- `recurse`: Properties of recursive Settings files, valid only for directories
- `src`: Source file to linked, only if `state=link`
- `dest`: Target symbol of linked file, only if `state=link`
- `state`: 
    - `directory`: Create if directory not exist
    - `file`: Default value, do operate to file, but never create new file
    - `link`: Create soft file link
    - `hard`: Create hard file link
    - `touch`: Create new empty file, if file exist, update create time
    - `absent`: Delete file, unlink file or delete directory

#### 1.10.1. Create file

In [ ]:
# Create empty file (use touch)
ansible group_debian1_vault -m file \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "path='/home/alvin/test.txt' \
        mode='0644' \
        state='touch'";
        
# Check created file
ansible group_debian1_vault -m raw \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "ls -al /home/alvin/test.*";
    
# Touch file again, and modify file mode
ansible group_debian1_vault -m file \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "path='/home/alvin/test.txt' \
        mode='u+rw,g-gw,o-gw' \
        state='touch'";
        
# Check created file again
ansible group_debian1_vault -m raw \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "ls -al /home/alvin/test.*";     

# Modify access time and modification time
ansible group_debian1_vault -m file \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "path='/home/alvin/test.txt' \
        modification_time='now' \
        access_time='202010010000.0'";

# Check created file again
ansible group_debian1_vault -m raw \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "ls -al /home/alvin/test.*";

#### 1.10.2. Change file mode

In [ ]:
# Change file owner, group and permission
ansible group_debian1_vault -m file \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "path='/home/alvin/test.txt' \
        owner='alvin' \
        group='alvin' \
        mode='0644'";
        
# Check file
ansible group_debian1_vault -m raw \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "ls -al /home/alvin/test.*";

#### 1.10.3. Make file link

In [ ]:
# Create soft file link
ansible group_debian1_vault -m file \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "src='/home/alvin/test.txt' \
        dest='/home/alvin/test-slink.txt' \
        owner='alvin' \
        group='alvin' \
        state='link'";

# Create hard file link
ansible group_debian1_vault -m file \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "src='/home/alvin/test.txt' \
        dest='/home/alvin/test-hlink.txt' \
        owner='alvin' \
        group='alvin' \
        state='link'";

# Set source file content
ansible group_debian1_vault -m copy \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "content='Hello World' \
        dest='/home/alvin/test.txt' \
        force='yes'";

# Check file link
ansible group_debian1_vault -m raw \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "cat /home/alvin/test-slink.txt";
    
# Check file link
ansible group_debian1_vault -m raw \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "cat /home/alvin/test-hlink.txt";

#### 1.10.4. Create and delete directory

In [ ]:
# Create directory
ansible group_debian1_vault -m file \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "path='/home/alvin/test_dir' \
        mode='0755' \
        state='directory'";
        
# Delete directory
ansible group_debian1_vault -m file \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "path='/home/alvin/test_dir' \
        state='absent'";

#### 1.10.5. Delete file

In [ ]:
ansible group_debian1_vault -m file \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "path='/home/alvin/test.txt' \
        state='absent'";

ansible group_debian1_vault -m file \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "path='/home/alvin/test-slink.txt' \
        state='absent'"

ansible group_debian1_vault -m file \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "path='/home/alvin/test-hlink.txt' \
        state='absent'"